In [ ]:
import bcrypt
import csv
import re
import requests

# File path for user credentials storage
USER_FILE = 'users.csv'
MAX_LOGIN_ATTEMPTS = 5

# Helper function to hash passwords
def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

# Helper function to check if password is valid
def check_password(hashed_password, password):
    return bcrypt.checkpw(password.encode('utf-8'), hashed_password.encode('utf-8'))

# Validate email format
def is_valid_email(email):
    email_regex = r'^\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return re.match(email_regex, email) is not None

# Validate password complexity
def is_valid_password(password):
    if (len(password) < 8 or
        not re.search(r'[A-Z]', password) or
        not re.search(r'[a-z]', password) or
        not re.search(r'[0-9]', password) or
        not re.search(r'[\W]', password)):
        return False
    return True

# Load user data from the CSV file
def load_users():
    users = {}
    try:
        with open(USER_FILE, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                users[row['email']] = {
                    'hashed_password': row['hashed_password'],
                    'security_question': row['security_question'],
                }
    except FileNotFoundError:
        with open(USER_FILE, mode='w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['email', 'hashed_password', 'security_question'])
            writer.writeheader()
    return users

# Save user data to the CSV file
def save_user(email, hashed_password, security_question):
    with open(USER_FILE, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['email', 'hashed_password', 'security_question'])
        writer.writerow({'email': email, 'hashed_password': hashed_password, 'security_question': security_question})

# Register a new user
def register_user():
    email = input("Enter your email: ")
    if not is_valid_email(email):
        print("Invalid email format. Please try again.")
        return
    
    password = input("Enter your password: ")
    if not is_valid_password(password):
        print("Password must be at least 8 characters long and include upper/lowercase letters, numbers, and special characters.")
        return

    security_question = input("Enter a security question: ")
    hashed_password = hash_password(password).decode('utf-8')
    
    save_user(email, hashed_password, security_question)
    print("User registered successfully!")

# User login
def login_user():
    users = load_users()
    attempts = 0
    while attempts < MAX_LOGIN_ATTEMPTS:
        email = input("Enter your email: ")
        password = input("Enter your password: ")

        if email in users and check_password(users[email]['hashed_password'], password):
            print("Login successful!")
            return True
        else:
            attempts += 1
            print(f"Invalid credentials. You have {MAX_LOGIN_ATTEMPTS - attempts} attempts left.")
    
    print("Maximum login attempts exceeded. Exiting...")
    return False

# Password recovery
def recover_password():
    users = load_users()
    email = input("Enter your registered email: ")

    if email in users:
        security_answer = input(f"Security Question: {users[email]['security_question']}\nAnswer: ")
        if security_answer:  # Here, you can implement actual answer checking if needed
            new_password = input("Enter your new password: ")
            if is_valid_password(new_password):
                hashed_password = hash_password(new_password).decode('utf-8')
                # Update the CSV file
                rows = []
                with open(USER_FILE, mode='r', newline='') as file:
                    reader = csv.DictReader(file)
                    for row in reader:
                        if row['email'] == email:
                            row['hashed_password'] = hashed_password
                        rows.append(row)

                with open(USER_FILE, mode='w', newline='') as file:
                    writer = csv.DictWriter(file, fieldnames=['email', 'hashed_password', 'security_question'])
                    writer.writeheader()
                    writer.writerows(rows)

                print("Password reset successfully!")
                return
    else:
        print("Email not found.")

# Fetch game prices from CheapShark API
def fetch_game_prices(game_name):
    response = requests.get(f'https://www.cheapshark.com/api/1.0/deals?title={game_name}')
    if response.status_code == 200:
        data = response.json()
        
        if not data:
            print("No deals found for this game.")
            return

        for deal in data:
            print(f"Game Title: {deal.get('title')}")
            print(f"Store: {deal.get('storeID')}")
            print(f"Normal Price: ${deal.get('normalPrice')}")
            print(f"Sale Price: ${deal.get('salePrice')}")
            print(f"Savings: {deal.get('savings')}%")
            print(f"Deal Rating: {deal.get('dealRating')}")
            print(f"Link: {deal.get('dealURL')}")
            print("------------------------------")
    else:
        print("Failed to fetch data from CheapShark API.")


def main():
    while True:
        print("\nWelcome to the Digital PC Game Price Comparison Console Application")
        print("1. Register")
        print("2. Login")
        print("3. Forgot Password")
        print("4. Exit")
        choice = input("Select an option: ")

        if choice == '1':
            register_user()
        elif choice == '2':
            if login_user():
                game_name = input("Enter the name of the game you want to search for: ")
                fetch_game_prices(game_name)
        elif choice == '3':
            recover_password()
        elif choice == '4':
            print("Exiting the application. Goodbye!")
            break
        else:
            print("Invalid option. Please try again.")

if __name__ == "__main__":
    main()
